In [3]:
#잘안됨!


import numpy as np # 필요 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter
import pytagcloud
import plotly
import plotly.graph_objs as go

In [17]:
# 데이터 불러오기
news = pd.read_csv("./korean_news_topic_201711.CSV", engine='python')
news = news['토픽키워드'] # 키워드 추출
news = news.values.tolist()

tfidf = TfidfVectorizer(max_features=200, max_df=0.95, min_df=0) # tf-idf 기준으로 뉴스 별
bagOfWord = tfidf.fit_transform(news).toarray() # 상위 100개 단어 추출

vect = CountVectorizer() # bag of word 만들기
vect.fit(news)
#bagOfWord = vect.transform(news).toarray()
query = vect.transform([news[0]]).toarray()[0]

In [18]:
jaccard_sim =[] # jaccard similarity를 이용하여 쿼리와 문장의 유사도 계산
cosine_sim = [] # cosine similarity를 이용하여 쿼리와 문장의 유사도 계산
euclidean_dist = [] # euclidean dist를 이용하여 쿼리와 문장의 유사도 계산
pcc_sim = [] # PCC를 이용하여 쿼리와 문장의 유사도 계산
for temp_news in bagOfWord:
    jaccard_sim.append(jaccard_similarity_score(query, temp_news))
    cosine_sim.append(cosine_similarity([query], [temp_news])[0][0])
    euclidean_dist.append(euclidean_distances([query], [temp_news])[0][0])
    pcc_sim.append(np.corrcoef(query, temp_news)[0, 1])

ValueError: Found input variables with inconsistent numbers of samples: [8093, 200]

In [4]:
jaccard_sim = np.asarray(jaccard_sim) # similarity를 기반으로 정렬한 결과 저장
jaccard_sim = np.argsort(jaccard_sim)
jaccard_sim = jaccard_sim[::-1]

cosine_sim = np.asarray(cosine_sim)
cosine_sim = np.argsort(cosine_sim)
cosine_sim = cosine_sim[::-1] #역순을 위함 기본정렬이 오름차순이기 때문에

euclidean_dist = np.asarray(euclidean_dist)
euclidean_dist = np.argsort(euclidean_dist)

pcc_sim = np.asarray(pcc_sim) # similarity를 기반으로 정렬한 결과 저장
pcc_sim = np.argsort(pcc_sim)
pcc_sim = pcc_sim[::-1]

In [5]:
top10 = jaccard_sim[1:11] # jaccard similarity, Top 10 뉴스 특성 분석
top10_news = news[top10]

keywords =[]
for n in top10_news:
    keywords = keywords + n.split(',')

count = Counter(keywords) # 각 단어 별 개수 계산하기
tags = count.most_common(200) # 상위 200개 추출

word = []
counts = []

for key in tags:
    word.append(key[0])
    counts.append(key[1])

In [6]:
# histogram 만들기
data = [go.Bar(x=word, y=counts)]
filename = 'doc-histogram-jaccard'
plotly.offline.plot(data, filename=filename + '.html')

'file://C:\\Users\\user19\\tidus\\Similarity\\searchInfo\\doc-histogram-jaccard.html'

In [7]:
# wordcloud 만들기
taglist = pytagcloud.make_tags(tags, maxsize=30) 
pytagcloud.create_tag_image(taglist, 'doc-jaccard.jpg', fontname='Malgun Gothic', rectangular=False)